In [ ]:
import pandas as pd
import numpy as np


In [5]:
inventory_df = pd.read_excel(r"C:\Users\traju\OneDrive\Desktop\data\Hospital Analystics\Dataset_2\Updated_Datasets\Updated_Enhanced_Inventory_20250909_.xlsx")
patients_df = pd.read_excel(r"C:\Users\traju\OneDrive\Desktop\data\Hospital Analystics\Dataset_2\Updated_Datasets\Updated_Enhanced_Patients_20250909_.xlsx")
purchase_orders_df = pd.read_excel(r"C:\Users\traju\OneDrive\Desktop\data\Hospital Analystics\Dataset_2\Updated_Datasets\Updated_Enhanced_Supply_Chain_20250909.xlsx")
deliveries_df = pd.read_excel(r"C:\Users\traju\OneDrive\Desktop\data\Hospital Analystics\Dataset_2\Updated_Datasets\Updated_Enhanced_Supply_Chain_20250909.xlsx")
departments_df = pd.read_excel(r"C:\Users\traju\OneDrive\Desktop\data\Hospital Analystics\Dataset_2\Updated_Datasets\Updated_Hospital_Dataset_20250909_193847.xlsx")
physicians_df = pd.read_excel(r"C:\Users\traju\OneDrive\Desktop\data\Hospital Analystics\Dataset_2\Updated_Datasets\Updated_Hospital_Dataset_20250909_193847.xlsx")
skus_df = pd.read_excel(r"C:\Users\traju\OneDrive\Desktop\data\Hospital Analystics\Dataset_2\Updated_Datasets\Updated_Hospital_Dataset_20250909_193847.xlsx")
vendors_df = pd.read_excel(r"C:\Users\traju\OneDrive\Desktop\data\Hospital Analystics\Dataset_2\Updated_Datasets\Updated_Hospital_Dataset_20250909_193847.xlsx")
transactions_df = pd.read_excel(r"C:\Users\traju\OneDrive\Desktop\data\Hospital Analystics\Dataset_2\Updated_Datasets\Updated_Enhanced_Transactions_2024_20250909.xlsx")


In [41]:

# Inventory Stock Status - Count of items for each stock_status
print(" Inventory Stock Status: ")
stock_status_distribution = inventory_df['stock_status'].value_counts()
print("\nDistribution of Inventory Stock Status:")
print(stock_status_distribution)
print("-" * 50)

 Inventory Stock Status: 

Distribution of Inventory Stock Status:
stock_status
Critical_Low    641
Low             280
Adequate        123
Normal          108
Overstocked      45
Name: count, dtype: int64
--------------------------------------------------


In [42]:

# Patient Demographics - Distribution of age_group and patient_type
print(" Patient Demographics: ")
patient_age_distribution = patients_df['age_group'].value_counts()
patient_type_distribution = patients_df['patient_type'].value_counts()
print("\nDistribution of Patient Age Groups:")
print(patient_age_distribution)
print("\nDistribution of Patient Types:")
print(patient_type_distribution)
print("-" * 50)

 Patient Demographics: 

Distribution of Patient Age Groups:
age_group
19-35    14909
0-18     12331
36-50    10042
51-65     7677
65+       5041
Name: count, dtype: int64

Distribution of Patient Types:
patient_type
Inpatient     20125
Emergency     15010
Outpatient    14865
Name: count, dtype: int64
--------------------------------------------------


In [43]:

#  Supply Chain Urgency - Distribution of urgency_level in purchase orders
print(" Supply Chain Urgency: ")
po_urgency_distribution = purchase_orders_df['urgency_level'].value_counts()
print("\nDistribution of Urgency Levels in Purchase Orders:")
print(po_urgency_distribution)
print("-" * 50)

 Supply Chain Urgency: 

Distribution of Urgency Levels in Purchase Orders:
urgency_level
Normal       8667
Urgent       2418
Emergency    1343
Name: count, dtype: int64
--------------------------------------------------


In [44]:

# Transaction Cost Analysis - Total and average cost by transaction type
print(" Transaction Cost Analysis: ")
transaction_cost_analysis = transactions_df.groupby('transaction_type')['total_cost'].agg(['sum', 'mean', 'median']).sort_values(by='sum', ascending=False)
print("\nTotal and Average Cost by Transaction Type:")
print(transaction_cost_analysis)
print("-" * 50)

 Transaction Cost Analysis: 

Total and Average Cost by Transaction Type:
                           sum        mean  median
transaction_type                                  
Prescription      1.023923e+08  238.738492   124.6
Procedure         3.743370e+07  308.680648   216.2
--------------------------------------------------


In [45]:

# Bounced Transaction Analysis - Total revenue lost and common bounce reasons
print("Insight 8: Bounced Transaction Analysis: ")
bounced_transactions = transactions_df[transactions_df['bounced'] == True]
total_revenue_lost = bounced_transactions['revenue_lost'].sum()
bounce_reason_counts = bounced_transactions['bounce_reason'].value_counts()
print(f"\nTotal Revenue Lost from Bounced Transactions: ${total_revenue_lost:,.2f}")
print("\nMost Common Bounce Reasons:")
print(bounce_reason_counts)
print("-" * 50)


Insight 8: Bounced Transaction Analysis: 

Total Revenue Lost from Bounced Transactions: $17,074,983.38

Most Common Bounce Reasons:
bounce_reason
High Cost                 23366
Insurance Issues          14688
Patient Declined          11755
Alternative Prescribed     8873
Name: count, dtype: int64
--------------------------------------------------


In [53]:

# Convert 'admission_date' and 'discharge_date' columns to datetime objects
patients_df['admission_date'] = pd.to_datetime(patients_df['admission_date'])
patients_df['discharge_date'] = pd.to_datetime(patients_df['discharge_date'])

# Calculate the length of stay in days
patients_df['calculated_length_of_stay'] = (patients_df['discharge_date'] - patients_df['admission_date']).dt.days


# Display a sample of the data to show the result
print("\nSample of data showing original and calculated length of stay:")
print(patients_df[['patient_id', 'admission_date', 'discharge_date', 'length_of_stay', 'calculated_length_of_stay']].head())

# Perform a quick check to see if the calculated and original values are consistent
are_consistent = (patients_df['length_of_stay'] - patients_df['calculated_length_of_stay']).abs().max() < 1
print(f"\nConsistency check between original and calculated length of stay: {are_consistent}")


Sample of data showing original and calculated length of stay:
  patient_id admission_date      discharge_date  length_of_stay  \
0   PT000700     2023-01-01 2023-01-13 12:00:00            12.5   
1   PT001116     2023-01-01 2023-01-03 00:00:00             2.0   
2   PT001636     2023-01-01 2023-01-02 16:48:00             1.7   
3   PT001698     2023-01-01 2023-01-01 12:00:00             0.5   
4   PT002638     2023-01-01 2023-01-09 09:36:00             8.4   

   calculated_length_of_stay  
0                         12  
1                          2  
2                          1  
3                          0  
4                          8  

Consistency check between original and calculated length of stay: True


In [46]:

# Patient Financials by Demographics (Insurance and Age)
print(" Patient Financials by Demographics: ")
patient_transactions = pd.merge(transactions_df, patients_df, on='patient_id', how='left')
financial_by_demographics = patient_transactions.groupby(['insurance_type', 'age_group'])['total_cost'].sum().unstack()
print(financial_by_demographics)
print("-" * 50)


 Patient Financials by Demographics: 
age_group              0-18        19-35        36-50        51-65  \
insurance_type                                                       
Government      14781220.79  23179219.05  17523338.23  15249475.77   
Private          7699053.78  12196214.92   8613061.46   7508227.00   
Self_Pay         2290219.19   3793915.58   2817051.01   2845643.51   

age_group               65+  
insurance_type               
Government      13434894.77  
Private          5973060.88  
Self_Pay         1921419.37  
--------------------------------------------------


In [47]:

# Correlation between Inventory Turnover and Expiry
print(" Correlation between Inventory Turnover and Expiry:")
turnover_expiry_correlation = inventory_df[['turnover_rate', 'days_to_expiry']].corr().iloc[0, 1]
print(f"Correlation between Inventory Turnover Rate and Days to Expiry: {turnover_expiry_correlation:.2f}")
print("-" * 50)

 Correlation between Inventory Turnover and Expiry:
Correlation between Inventory Turnover Rate and Days to Expiry: 0.13
--------------------------------------------------
